In [ ]:
import torch
import torch.nn as nn

## MNIST 훈련

In [ ]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [ ]:
# 랜덤패턴
# 1 7 4 5 7 3 4 5 6 7 8 0 19 14 53 43 23 12 5 2

고정된 무작위성의 필요성
- 매 시행마다 다른 랜덤패턴을 사용한다고 가정
    - 1회차 : 배치 32, 랜덤패턴 1 사용 - 결과 0.98
    - 2회차 : 배치 128, 랜덤패턴 2 사용 - 결과 0.97   
결과에 랜덤패턴이 영향을 미칠 수 있으므로, 랜덤패턴은 고정된 값을 사용하는 것이 좋다.

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 랜덤 시드 고정 : 고정된 무작위성
torch.manual_seed(777)

# GPU 사용 가능일 경우 랜덤 시드 고정
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [ ]:
learning_rate = 0.001
training_epochs = 15
batch_size = 256

In [ ]:
mnist_train = dsets.MNIST(root='MNIST_data/', # 다운로드 경로 지정
                          train=True, # True를 지정하면 훈련 데이터로 다운로드
                          transform=transforms.ToTensor(), # 텐서로 변환
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/', # 다운로드 경로 지정
                         train=False, # False를 지정하면 테스트 데이터로 다운로드
                         transform=transforms.ToTensor(), # 텐서로 변환
                         download=True)

In [ ]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [ ]:
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()

        self.flatten = nn.Flatten()
        self.hidden_layer = nn.Sequential(
            nn.Linear(784, 256),
            nn.ReLU(),
            nn.Linear(256, 256),
            nn.ReLU(),
        )

        self.output_layer = nn.Sequential(
            nn.Linear(256, 10)
        )

    # 순전파 연산
    def forward(self, x):
        x = self.flatten(x)
        x = self.hidden_layer(x)
        x = self.output_layer(x)
        return x

In [ ]:
# CNN 모델 정의
model = MyModel().to(device) # .to('cuda') == .cuda() / .to('cpu') == .cpu()

Loss
- pytorch에서는 CrossEntropyLoss()로 sparse categorical, categorical 두개를 자동으로 인식하여 처리한다. (softmax 포함하기 때문에 모델에서 Softmax 생략)
- 이진분류 경우에는 BCELoss() 사용 (sigmoid 미포함이므로 모델에서 sigmoid 선언)

In [ ]:
criterion = torch.nn.CrossEntropyLoss()   # 비용 함수에 소프트맥스 함수 포함되어져 있음.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
total_batch = len(data_loader)
print('총 배치의 수 : {}'.format(total_batch))

총 배치의 수 : 234


In [ ]:
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader: # 미니 배치 단위로 꺼내온다. X는 미니 배치, Y는 레이블.
        # image is already size of (28x28), no reshape
        # label is not one-hot encoded
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad() # gradient 초기화
        hypothesis = model(X) # forward 실행
        cost = criterion(hypothesis, Y) # CrossEntropyLoss
        cost.backward() # gradient 계산
        optimizer.step() # weight 업데이트

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

[Epoch:    1] cost = 0.406873494
[Epoch:    2] cost = 0.154265001
[Epoch:    3] cost = 0.104517683
[Epoch:    4] cost = 0.0746622086
[Epoch:    5] cost = 0.0589870699
[Epoch:    6] cost = 0.0457189679
[Epoch:    7] cost = 0.0369828716
[Epoch:    8] cost = 0.0269247498
[Epoch:    9] cost = 0.0210895967
[Epoch:   10] cost = 0.0190479066
[Epoch:   11] cost = 0.0155847874
[Epoch:   12] cost = 0.012894908
[Epoch:   13] cost = 0.00870932639
[Epoch:   14] cost = 0.00537019176
[Epoch:   15] cost = 0.00682783592


In [ ]:
# 학습을 진행하지 않을 것이므로 torch.no_grad()
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

Accuracy: 0.9787999987602234


#### 문제
직접 해봅시다.

위 코드를 참고하여, 레이어를 쌓아보고, 학습시켜봅시다.

In [3]:
import torch
import torch.nn as nn

In [4]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 랜덤 시드 고정 : 고정된 무작위성
torch.manual_seed(777)

# GPU 사용 가능일 경우 랜덤 시드 고정
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [6]:
learning_rate = 0.001
training_epochs = 5
batch_size = 256

In [7]:
mnist_train = dsets.MNIST(root='MNIST_data/', # 다운로드 경로 지정
                          train=True, # True를 지정하면 훈련 데이터로 다운로드
                          transform=transforms.ToTensor(), # 텐서로 변환
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/', # 다운로드 경로 지정
                         train=False, # False를 지정하면 테스트 데이터로 다운로드
                         transform=transforms.ToTensor(), # 텐서로 변환
                         download=True)

100%|██████████| 9.91M/9.91M [00:00<00:00, 18.0MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 483kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.43MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 8.77MB/s]


In [20]:
learning_rate = 0.01
training_epochs = 15
batch_size = 128

In [21]:
mnist_train = dsets.MNIST(root='MNIST_data/', # 다운로드 경로 지정
                          train=True, # True를 지정하면 훈련 데이터로 다운로드
                          transform=transforms.ToTensor(), # 텐서로 변환
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/', # 다운로드 경로 지정
                         train=False, # False를 지정하면 테스트 데이터로 다운로드
                         transform=transforms.ToTensor(), # 텐서로 변환
                         download=True)

In [22]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [23]:
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()

        self.flatten = nn.Flatten()
        self.hidden_layer = nn.Sequential(
            nn.Linear(784, 256),
            nn.ReLU(),
            nn.Linear(256, 256),
            nn.ReLU(),
        )

        self.output_layer = nn.Sequential(
            nn.Linear(256, 10)
        )

    # 순전파 연산
    def forward(self, x):
        x = self.flatten(x)
        x = self.hidden_layer(x)
        x = self.output_layer(x)
        return x

In [24]:
# CNN 모델 정의
model = MyModel().to(device) # .to('cuda') == .cuda() / .to('cpu') == .cpu()

In [25]:
criterion = torch.nn.CrossEntropyLoss()   # 비용 함수에 소프트맥스 함수 포함되어져 있음.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [26]:
total_batch = len(data_loader)
print('총 배치의 수 : {}'.format(total_batch))

총 배치의 수 : 468


In [27]:
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader: # 미니 배치 단위로 꺼내온다. X는 미니 배치, Y는 레이블.
        # image is already size of (28x28), no reshape
        # label is not one-hot encoded
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad() # gradient 초기화
        hypothesis = model(X) # forward 실행
        cost = criterion(hypothesis, Y) # CrossEntropyLoss
        cost.backward() # gradient 계산
        optimizer.step() # weight 업데이트

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

[Epoch:    1] cost = 0.236168519
[Epoch:    2] cost = 0.126896158
[Epoch:    3] cost = 0.107409902
[Epoch:    4] cost = 0.101554103
[Epoch:    5] cost = 0.0942711309
[Epoch:    6] cost = 0.0798377469
[Epoch:    7] cost = 0.0754617453
[Epoch:    8] cost = 0.0758128166
[Epoch:    9] cost = 0.0650135949
[Epoch:   10] cost = 0.0630504861
[Epoch:   11] cost = 0.0700941384
[Epoch:   12] cost = 0.0652838871
[Epoch:   13] cost = 0.0528143011
[Epoch:   14] cost = 0.0669565201
[Epoch:   15] cost = 0.0567905419


In [28]:
# 학습을 진행하지 않을 것이므로 torch.no_grad()
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

Accuracy: 0.9695999622344971


/usr/local/lib/python3.11/dist-packages/torchvision/datasets/mnist.py:81: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.11/dist-packages/torchvision/datasets/mnist.py:71: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")
